In [ ]:
# 1. Gerekli Kütüphaneleri Kuralım
print("Kütüphaneler kuruluyor (imagehash, Pillow)...")
!pip install -q Pillow imagehash

# 2. Gerekli Kütüphaneleri Python'a Tanıtalım (Import Ettik)
print("Kütüphaneler import ediliyor...")
import os
import zipfile
import shutil
import imagehash
from PIL import Image
from google.colab import drive

# 3. Google Drive'ı Bağlayalım
try:
    drive.mount('/content/drive')
    print("Google Drive başarıyla bağlandı.")
except Exception as e:
    print(f"Drive bağlanırken hata oluştu veya zaten bağlı: {e}")

print("\n--- ADIM 1 TAMAMLANDI. ADIM 2'ye geçebilirsiniz. ---")

Kütüphaneler kuruluyor (imagehash, Pillow)...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 22.5 MB/s eta 0:00:00
Kütüphaneler import ediliyor...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive başarıyla bağlandı.

--- ADIM 1 TAMAMLANDI. ADIM 2'ye geçebilirsiniz. ---


In [ ]:
print("FAALİYET 1 (AŞAMA 1: VERİ TEMİZLEME) BAŞLATILDI")

# 1. Yolları Tanımlayalım (Türkçe karakterli doğru yolumuzu kullanalım)
drive_proje_yolu = "/content/drive/MyDrive/TUBITAK_Projesi/"
eski_zip_yolu = os.path.join(drive_proje_yolu, "dataset_eski.zip")
yeni_zip_yolu = os.path.join(drive_proje_yolu, "dataset_yeni.zip")

# Colab'da çalışacağımız geçici klasörler
eski_veri_klasoru = "/content/dataset_eski_raw"
yeni_veri_klasoru = "/content/dataset_yeni_raw"
temiz_veri_klasoru = "/content/dataset_temiz_613" # Nihai temiz resimler buraya gelecektir

# 2. Geçici Klasörleri Temizle (Önceki oturumlardan kalıntı varsa temizleriz)
print("Geçici klasörler temizleniyor...")
if os.path.exists(eski_veri_klasoru): shutil.rmtree(eski_veri_klasoru)
if os.path.exists(yeni_veri_klasoru): shutil.rmtree(yeni_veri_klasoru)
if os.path.exists(temiz_veri_klasoru): shutil.rmtree(temiz_veri_klasoru)
os.makedirs(eski_veri_klasoru)
os.makedirs(yeni_veri_klasoru)
os.makedirs(temiz_veri_klasoru)

# 3. Ham Veri ZIP'lerini Colab'a Açalım
try:
    print(f"'{eski_zip_yolu}' açılıyor...")
    with zipfile.ZipFile(eski_zip_yolu, 'r') as zip_ref:
        zip_ref.extractall(eski_veri_klasoru)
    print(f"'{yeni_zip_yolu}' açılıyor...")
    with zipfile.ZipFile(yeni_zip_yolu, 'r') as zip_ref:
        zip_ref.extractall(yeni_veri_klasoru)
    print("Ham veri başarıyla Colab ortamına açıldı.")
except Exception as e:
    print(f"KRİTİK HATA: ZIP dosyaları bulunamadı veya açılamadı. Hata: {e}")
    print("Lütfen Google Drive'da 'TUBITAK_Projesi' klasöründe 'dataset_eski.zip' ve 'dataset_yeni.zip' dosyalarının olduğundan emin olun.")
    exit() # Hata varsa script'i durdur diyelim

# 4. pHash Temizleme Script'i
gorulen_hashler = set()
eski_toplam = 0
yeni_toplam = 0
toplam_kopya = 0
toplam_temiz = 0

def process_images_in_folder(folder_path, is_base_dataset=True):
    """
    Bir klasördeki resimleri tarar, pHash'lerini hesaplar,
    kopyaları bulur ve temiz olanları 'temiz_veri_klasoru'ne kopyalar.
    """
    global eski_toplam, yeni_toplam, toplam_kopya, toplam_temiz

    print(f"\n'{folder_path}' klasörü taranıyor...")
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue # Resim dosyası değilse atla

            file_path = os.path.join(root, filename)

            if is_base_dataset: eski_toplam += 1
            else: yeni_toplam += 1

            try:
                with Image.open(file_path) as img:
                    hash_degeri = imagehash.phash(img)

                if hash_degeri in gorulen_hashler:
                    toplam_kopya += 1
                else:
                    gorulen_hashler.add(hash_degeri)
                    hedef_dosya_adi = f"{hash_degeri}.jpg"
                    hedef_yol = os.path.join(temiz_veri_klasoru, hedef_dosya_adi)
                    img.convert("RGB").save(hedef_yol, "JPEG")
                    toplam_temiz += 1
            except Exception as e:
                print(f"HATA: {filename} işlenemedi, atlanıyor. Hata: {e}")

# Script'i çalıştır
process_images_in_folder(eski_veri_klasoru, is_base_dataset=True)
process_images_in_folder(yeni_veri_klasoru, is_base_dataset=False)

# 5. Temiz Veriyi Drive'a Geri Kaydet
temiz_zip_adi = "dataset_temiz_v1.zip"
temiz_zip_yolu_colab = f"/content/{temiz_zip_adi}"
temiz_zip_yolu_drive = os.path.join(drive_proje_yolu, temiz_zip_adi)

print(f"\nTemizlenmiş {toplam_temiz} resim '{temiz_zip_adi}' olarak ZIP'leniyor...")
shutil.make_archive(temiz_zip_yolu_colab.replace('.zip', ''), 'zip', temiz_veri_klasoru)

print(f"Temiz ZIP dosyası Google Drive'a kopyalanıyor: {temiz_zip_yolu_drive}")
shutil.copy(temiz_zip_yolu_colab, drive_proje_yolu)
print("Kopyalama tamamlandı.")

# 6. NİHAİ RAPORUMUZ
print("\n--- FAALİYET 1 (AŞAMA 1): NİHAİ TEMİZLEME RAPORU ---")
print(f"Eski Veri Seti ('dataset_eski') Tarandı: {eski_toplam} resim")
print(f"Yeni Veri Seti ('dataset_yeni') Tarandı: {yeni_toplam} resim")
print(f"Toplam Ham Veri: {eski_toplam + yeni_toplam} resim (~694)")
print("---------------------------------")
print(f"Bulunan Toplam Kopya (Duplicate) Resim: {toplam_kopya} adet")
print(f"Nihai Temiz ve Eşsiz Resim Sayısı: {toplam_temiz} adet (613 olmalı)")
print(f"Temiz veri seti '{temiz_zip_adi}' olarak Google Drive'a kaydedildi.")
print("\nAŞAMA 1 BAŞARIYLA TAMAMLANDI.")

--- FAALİYET 1 (AŞAMA 1: VERİ TEMİZLEME) BAŞLATILDI ---
Geçici klasörler temizleniyor...
'/content/drive/MyDrive/TUBITAK_Projesi/dataset_eski.zip' açılıyor...
'/content/drive/MyDrive/TUBITAK_Projesi/dataset_yeni.zip' açılıyor...
Ham veri başarıyla Colab ortamına açıldı.

'/content/dataset_eski_raw' klasörü taranıyor...

'/content/dataset_yeni_raw' klasörü taranıyor...


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (171668516 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(



Temizlenmiş 683 resim 'dataset_temiz_v1.zip' olarak ZIP'leniyor...
Temiz ZIP dosyası Google Drive'a kopyalanıyor: /content/drive/MyDrive/TUBITAK_Projesi/dataset_temiz_v1.zip
Kopyalama tamamlandı.

--- FAALİYET 1 (AŞAMA 1): NİHAİ TEMİZLEME RAPORU ---
Eski Veri Seti ('dataset_eski') Tarandı: 423 resim
Yeni Veri Seti ('dataset_yeni') Tarandı: 271 resim
Toplam Ham Veri: 694 resim (~694)
---------------------------------
Bulunan Toplam Kopya (Duplicate) Resim: 11 adet
Nihai Temiz ve Eşsiz Resim Sayısı: 683 adet (613 olmalı)
Temiz veri seti 'dataset_temiz_v1.zip' olarak Google Drive'a kaydedildi.

✅ AŞAMA 1 BAŞARIYLA TAMAMLANDI.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import zipfile
import shutil
from PIL import Image
from google.colab import drive

print("FAALİYET 1 (AŞAMA 2: VERİ STANDARDİZASYON - 256x256) BAŞLATILDI")

# 1. Ortamı Kur (Drive'ın zaten bağlı olduğunu varsayıyoruz)
# Gerekli kütüphaneler bir önceki hücrede zaten import edildi.
print("Ortam hazır.")

# 2. Yolları Tanımla
drive_proje_yolu = "/content/drive/MyDrive/TUBITAK_Projesi/"

# Çekilecek temiz veri (613 resim)
temiz_zip_yolu = os.path.join(drive_proje_yolu, "dataset_temiz_v1.zip")

# Colab'da ZIP'i açacağımız geçici klasör
temiz_veri_klasoru = "/content/dataset_temiz_613_ready"

# Standardizasyon (256x256) sonrası resimlerin konulacağı geçici klasör
son_veri_klasoru = "/content/dataset_son_hali_256"

# Drive'a kaydedilecek nihai ZIP dosyasının adı
nihai_zip_adi = "dataset_son_hali_256_v1.zip"
nihai_zip_yolu_colab = f"/content/{nihai_zip_adi}"
nihai_zip_yolu_drive = os.path.join(drive_proje_yolu, nihai_zip_adi)

# 3. Geçici Klasörleri Temizle
print("Geçici klasörler temizleniyor.")
if os.path.exists(temiz_veri_klasoru): shutil.rmtree(temiz_veri_klasoru)
if os.path.exists(son_veri_klasoru): shutil.rmtree(son_veri_klasoru)
os.makedirs(temiz_veri_klasoru)
os.makedirs(son_veri_klasoru)

# 4. Temiz Veriyi (613 Resim) ZIP'ten Aç
print(f"Temiz veri seti '{temiz_zip_yolu}' Colab ortamına açılıyor...")
if os.path.exists(temiz_zip_yolu):
    try:
        with zipfile.ZipFile(temiz_zip_yolu, 'r') as zip_ref:
            zip_ref.extractall(temiz_veri_klasoru)
        print("Veri açma başarılı. Standardizasyon başlıyor...")
    except Exception as e:
        print(f"KRİTİK HATA: '{temiz_zip_yolu}' dosyası bozuk veya açılamıyor. Hata: {e}")
        exit() # Hata varsa script'i durdur
else:
    print(f"KRİTİK HATA: '{temiz_zip_yolu}' bulunamadı. Lütfen Google Drive'da dosyanın varlığını kontrol edin.")
    exit() # Hata varsa script'i durdur

# 5. STANDARDİZASYON (256x256 KIRPMA)
hedef_boyut = 256
islenen_resim_sayisi = 0
hata_alan_resim_sayisi = 0

dosya_listesi = os.listdir(temiz_veri_klasoru)
print(f"Toplam {len(dosya_listesi)} dosya bulundu, standardizasyon başlıyor...")

for filename in dosya_listesi:
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue # Sadece resimleri işle

    file_path = os.path.join(temiz_veri_klasoru, filename)

    try:
        with Image.open(file_path) as img:
            img = img.convert("RGB") # Modu 'RGB' yap (StyleGAN için şart olduğu için )

            # En boy oranını koruyarak yeniden boyutlandırma
            width, height = img.size
            if width > height:
                new_height = hedef_boyut
                new_width = int(width * (hedef_boyut / height))
            else:
                new_width = hedef_boyut
                new_height = int(height * (hedef_boyut / width))

            img_resized = img.resize((new_width, new_height), Image.LANCZOS)

            # Ortadan Kırpma (Center Crop)
            left = (new_width - hedef_boyut) / 2
            top = (new_height - hedef_boyut) / 2
            right = (new_width + hedef_boyut) / 2
            bottom = (new_height + hedef_boyut) / 2

            img_cropped = img_resized.crop((left, top, right, bottom))

            # Sonucu kaydet
            hedef_yol = os.path.join(son_veri_klasoru, filename)
            img_cropped.save(hedef_yol, "JPEG")
            islenen_resim_sayisi += 1

    except Exception as e:
        # Hatalı/Bozuk resimleri atla
        hata_alan_resim_sayisi += 1
        print(f"HATA: {filename} işlenemedi, atlanıyor. Hata: {e}")

# 6. Nihai Veri Setini Drive'a Geri Kaydet Diyelim
print("\nStandardizasyon tamamlandı.")
print(f"Nihai (256x256) veri seti '{nihai_zip_adi}' olarak ZIP'leniyor...")
shutil.make_archive(nihai_zip_yolu_colab.replace('.zip', ''), 'zip', son_veri_klasoru)

print(f"Nihai ZIP dosyası Google Drive'a kopyalanıyor: {nihai_zip_yolu_drive}")
shutil.copy(nihai_zip_yolu_colab, drive_proje_yolu)
print("Kopyalama tamamlandı.")

# 7. NİHAİ RAPORUMUZ
print("\n--- FAALİYET 1 (AŞAMA 2): NİHAİ STANDARDİZASYON RAPORU ---")
print(f"Başarıyla İşlenen ve Standartlaştırılan (256x256) Resim Sayısı: {islenen_resim_sayisi} adet")
print(f"Atlanan Hatalı Resim Sayısı: {hata_alan_resim_sayisi} adet")
print(f"Nihai veri setiniz '{nihai_zip_adi}' olarak Google Drive'a kaydedildi.")
print("\nFAALİET 1 (HER İKİ AŞAMA DA) TAMAMLANDI.")

--- FAALİYET 1 (AŞAMA 2: VERİ STANDARDİZASYON - 256x256) BAŞLATILDI ---
Ortam hazır.
Geçici klasörler temizleniyor...
Temiz veri seti '/content/drive/MyDrive/TUBITAK_Projesi/dataset_temiz_v1.zip' Colab ortamına açılıyor...
Veri açma başarılı. Standardizasyon başlıyor...
Toplam 683 dosya bulundu, standardizasyon başlıyor...

Standardizasyon tamamlandı.
Nihai (256x256) veri seti 'dataset_son_hali_256_v1.zip' olarak ZIP'leniyor...
Nihai ZIP dosyası Google Drive'a kopyalanıyor: /content/drive/MyDrive/TUBITAK_Projesi/dataset_son_hali_256_v1.zip
Kopyalama tamamlandı.

--- FAALİYET 1 (AŞAMA 2): NİHAİ STANDARDİZASYON RAPORU ---
Başarıyla İşlenen ve Standartlaştırılan (256x256) Resim Sayısı: 683 adet
Atlanan Hatalı Resim Sayısı: 0 adet
Nihai veri setiniz 'dataset_son_hali_256_v1.zip' olarak Google Drive'a kaydedildi.

✅✅✅ FAALİET 1 (HER İKİ AŞAMA DA) TAMAMLANDI. ✅✅✅
